In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from common.util import preprocess, create_co_matrix, cos_similarity, most_similar, ppmi

In [3]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
c = create_co_matrix(corpus, vocab_size)

c0 = c[word_to_id["you"]]
c1 = c[word_to_id["i"]]
cos_similarity(c0, c1)

np.float64(0.7071067691154799)

In [4]:
most_similar("you", word_to_id, id_to_word, c, top=5)


[query] you
 goodbye: 0.7071067691154799
 hello: 0.7071067691154799
 i: 0.7071067691154799
 and: 0.0
 say: 0.0


In [5]:
most_similar("i", word_to_id, id_to_word, c, top=5)


[query] i
 goodbye: 0.9999999858578643
 you: 0.7071067691154799
 hello: 0.49999999292893216
 and: 0.0
 say: 0.0


In [6]:
W = ppmi(c)
W

array([[0.       , 1.8073549, 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [1.8073549, 0.       , 0.8073549, 0.       , 0.8073549, 0.8073549,
        0.       ],
       [0.       , 0.8073549, 0.       , 1.8073549, 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 1.8073549, 0.       , 1.8073549, 0.       ,
        0.       ],
       [0.       , 0.8073549, 0.       , 1.8073549, 0.       , 0.       ,
        0.       ],
       [0.       , 0.8073549, 0.       , 0.       , 0.       , 0.       ,
        2.807355 ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 2.807355 ,
        0.       ]], dtype=float32)

In [7]:
from dataset import ptb

In [11]:
window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = ptb.load_data("train")
vocab_size = len(word_to_id)

C = create_co_matrix(corpus, vocab_size, window_size)
W = ppmi(C, verbose=True)

try:
    from sklearn.utils.extmath import randomized_svd
    U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5, random_state=None)
    
except ImportError:
    U, S, V = np.linalg.svd(W)
    
word_vecs = U[:, :wordvec_size]

querys = ["you", "year", "car", "toyota"]
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 i: 0.6658942103385925
 anybody: 0.5775973200798035
 we: 0.5683689117431641
 do: 0.5625555515289307
 somebody: 0.5611586570739746

[query] year
 month: 0.676778256893158
 quarter: 0.6196840405464172
 earlier: 0.6148099303245544
 last: 0.6083329319953918
 next: 0.5821052193641663

[query] car
 luxury: 0.610798180103302
 auto: 0.5984219908714294
 lexus: 0.5400359630584717
 truck: 0.5332613587379456
 vehicle: 0.5123125910758972

[query] toyota
 motor: 0.7103665471076965
 nissan: 0.6871981024742126
 honda: 0.6790011525154114
 motors: 0.6576974987983704
 lexus: 0.6052846908569336
